In [1]:
import autograd

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

In [2]:
def advect(f, vx, vy):
  """Instead of moving the cell centers forward in time using the velocity fields,
  we look for the particles which end up exactly at the cell centers by tracing
  backwards in time from the cell centers. See 'implicit Euler integration.'"""
  rows, cols = f.shape
  cell_xs, cell_ys = np.meshgrid(np.arange(cols), np.arange(rows))
  center_xs = (cell_xs - vx).ravel()  # look backwards one timestep
  center_ys = (cell_ys - vy).ravel()

  left_ix = np.floor(center_ys).astype(int)  # get locations of source cells.
  top_ix  = np.floor(center_xs).astype(int)
  rw = center_ys - left_ix              # relative weight of cells on the right
  bw = center_xs - top_ix               # same for cells on the bottom
  left_ix  = np.mod(left_ix,     rows)  # wrap around edges of simulation.
  right_ix = np.mod(left_ix + 1, rows)
  top_ix   = np.mod(top_ix,      cols)
  bot_ix   = np.mod(top_ix  + 1, cols)

  # a linearly-weighted sum of the 4 cells closest to the source of the cell center.
  flat_f = (1 - rw) * ((1 - bw)*f[left_ix,  top_ix] + bw*f[left_ix,  bot_ix]) \
                + rw * ((1 - bw)*f[right_ix, top_ix] + bw*f[right_ix, bot_ix])
  return np.reshape(flat_f, (rows, cols))
  
def project(vx, vy, occlusion, width=0.4):
  """Project the velocity field to be approximately mass-conserving. Technically
  we are finding an approximate solution to the Poisson equation."""
  p = np.zeros(vx.shape)
  div = -0.5 * (np.roll(vx, -1, axis=1) - np.roll(vx, 1, axis=1)
              + np.roll(vy, -1, axis=0) - np.roll(vy, 1, axis=0))
  div = filter(div, occlusion, width=width)

  for k in range(50):  # use gauss-seidel to approximately solve linear system
      p = (div + np.roll(p, 1, axis=1) + np.roll(p, -1, axis=1)
                + np.roll(p, 1, axis=0) + np.roll(p, -1, axis=0))/4.0
      p = filter(p, occlusion, width=width)

  vx = vx - 0.5*(np.roll(p, -1, axis=1) - np.roll(p, 1, axis=1))
  vy = vy - 0.5*(np.roll(p, -1, axis=0) - np.roll(p, 1, axis=0))

  vx = occlude(vx, occlusion)
  vy = occlude(vy, occlusion)
  return vx, vy

def simulate(vx, vy, smoke, num_time_steps, ax=None, render=False):
    print("Running simulation...")
    for t in range(num_time_steps):
        if ax: plot_matrix(ax, smoke, t, render)
        vx_updated = advect(vx, vx, vy)
        vy_updated = advect(vy, vx, vy)
        vx, vy = project(vx_updated, vy_updated)
        smoke = advect(smoke, vx, vy)
    if ax: plot_matrix(ax, smoke, num_time_steps, render)
    return smoke

In [ ]:
u, v = np.random.randn(x), np.random.randn(*x.shape)
[u_rot, v_rot, u_irr, v_irr] = [gaussian_filter(v, 2) for v in [u_rot, v_rot, u_irr, v_irr]]

In [ ]:
init_dx_and_dy = np.zeros((2, rows, cols)).ravel()

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, frameon=False)
    
simulate(init_vx, init_vy, init_smoke, simulation_timesteps, ax)